### Load tensorflow

In [1]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Collect Data
<font size="2">Download data from Project Gutenberg site -> http://www.gutenberg.org/files/1342/1342-0.txt </font>

In [2]:
book_text = open('1342-0.txt', encoding='utf8').read()
print('Length of the book: ' , len(book_text))

Length of the book:  704190


In [3]:
book_text[1000:1100]

'ful property\nof some one or other of their daughters.\n\n“My dear Mr. Bennet,” said his lady to him on'

### Tokenize the data

In [4]:
#Tokenize at character level
t = tf.keras.preprocessing.text.Tokenizer(char_level=True)

#Fit tokenizer on the book
t.fit_on_texts(book_text)

#Vocablury size
vocab_size = len(t.word_index)
print('Number of unique characters: ', vocab_size)

Number of unique characters:  60


In [5]:
#Convert characters in the book to Numbers
book_num = t.texts_to_sequences(book_text)

In [6]:
#Build a dictionary which can convert numbers into chars
int_to_char = dict((i,c) for c, i in t.word_index.items()) 

### Batch Generator

In [7]:
import numpy as np

In [8]:
record_num = 0 #starting batch number
sequence_length = 100 #Length of input sequence

def batch_generator(batch_size=32):
    
    #Will update batch_num
    global record_num    
    
    while True:
        
        #Empty list for input and output data
        input_data = []
        output_data = []

        for i in range(batch_size):

            #input sequence
            input_seq = book_num[(record_num * sequence_length) : (record_num * sequence_length) + sequence_length]
            #Output sequence
            output_seq = book_num[(record_num * sequence_length) + sequence_length]

            input_data.append(input_seq)
            output_data.append(output_seq)
            
            record_num = record_num + 1
            
            if((record_num*sequence_length + sequence_length) > len(book_num)):
                record_num = 0
                

        #Input data one hot encoding
        input_data = tf.keras.utils.to_categorical(input_data,num_classes=vocab_size+1)

        #Output data one hot encoding
        output_data = tf.keras.utils.to_categorical(output_data,num_classes=vocab_size+1)

        #Reshape input data into 3 dimensional numpy array
        #batch_size x sequence_length x vocab_size+1
        input_data = np.reshape(input_data,
                                (len(input_data),
                                 sequence_length,
                                 vocab_size+1))
        
        yield input_data, output_data

### Build the Model

In [9]:
model = tf.keras.models.Sequential()

#LSTM
model.add(tf.keras.layers.LSTM(256, input_shape=(sequence_length,vocab_size+1)))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(vocab_size+1, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy') #No accuracy tracking here

### Print model output during Training

In [10]:
import numpy as np

In [11]:
#Identify a random sequence which we will use to generate output
start_pos = np.random.randint(0, high=(len(book_num) - sequence_length))
test_seq =  book_num[start_pos : start_pos+sequence_length]

In [12]:
#Print random starting sequence for prediction
print('Initial sequence is: ')
for i in range (sequence_length):
    print(int_to_char[test_seq[i][0]], end='')

Initial sequence is: 
nd censure of the ladies both of netherfield and
rosings.

their visitors stayed with them above hal

### Prediction function

In [13]:
def predict_seq(epoch, logs):
    
    print('\n\nOutput sequence after epoch ', epoch, ' :')
    
    #Initialize predicted output
    predicted_output = ''
    
    #lets predict 50 next chars
    current_seq = np.copy(test_seq)
    
    for i in range(50):
        current_seq_one_hot = tf.keras.utils.to_categorical(current_seq, num_classes=vocab_size+1)
        
        data_input = np.reshape(current_seq_one_hot,(1,
                                                     current_seq_one_hot.shape[0],
                                                     current_seq_one_hot.shape[1]))
        
        #Get the char int with maximum probability
        predicted_char_int = np.argmax(model.predict(data_input)[0])
        
        if (predicted_char_int != 0):
            
            #Add to the predicted out, convert int to char
            predicted_output = predicted_output + int_to_char[predicted_char_int]
        
        #Update seq with new value at the end
        current_seq = np.roll(current_seq, -1)
        current_seq[current_seq.shape[0]-1] = [predicted_char_int]
    
    print(predicted_output)

### Execute the model

In [14]:
#Create a LabdaCallback to do prediction at end of every epoch
lambda_checkpoint = tf.keras.callbacks.LambdaCallback(on_epoch_end=predict_seq)

#Create a model checkpoint to store model after each epoch if loss reduces
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('char_rnn.h5',
                                                      monitor='loss',
                                                      save_best_only=True)

In [ ]:
batch_size = 2000
train_generator = batch_generator(batch_size=batch_size)

#Fit generator
model.fit_generator(train_generator,
                    epochs=10,
                    steps_per_epoch = (len(book_num)- sequence_length)// batch_size,                    
                    callbacks=[model_checkpoint, lambda_checkpoint])

Epoch 1/10
351/352 [============================>.] - ETA: 0s - loss: 2.6616

Output sequence after epoch  0  :
 ansto sou the her ansere to the her ansere to the
352/352 [==============================] - 172s 489ms/step - loss: 2.6601
Epoch 2/10
351/352 [============================>.] - ETA: 0s - loss: 1.6164

Output sequence after epoch  1  :
leassoo

ol  anee the teetentton to mand as astere
352/352 [==============================] - 169s 482ms/step - loss: 1.6147
Epoch 3/10
351/352 [============================>.] - ETA: 0s - loss: 0.5572

Output sequence after epoch  2  :
llasssowd heccrr moffrilly teethen

iof betennne l
352/352 [==============================] - 169s 481ms/step - loss: 0.5562
Epoch 4/10
351/352 [============================>.] - ETA: 0s - loss: 0.1607

Output sequence after epoch  3  :
lassatthe who  ho  her eleenentooneaccallanntthe w
352/352 [==============================] - 170s 482ms/step - loss: 0.1605
Epoch 5/10
 39/352 [==>...........................] 